## RF_Key_Points

### 1. 集成学习概念
集成学习（Ensemble Learning）的目的是通过结合多个个体学习器的预测结果来改善单个学习器的**泛化能力**和**鲁棒性**。 <br>
一般来说集成学习可以分为三大类：
* 用于减少方差的bagging；
* 用于减少偏差的boosting；
* 用于提升预测结果的stacking；<br>

也可以根据个体学习器的生成方式分为两大类：
* 即个体学习器之间存在强依赖关系、必须串行生成的序列化方法，如Boosting and AdaBoost；
* 以及个体学习器间不存在强依赖关系、可同时生成的并行化方法，如Bagging and Random Forest。


### 2. 个体学习器概念
个体学习器是一个相对概念，在集成学习中，集成之前的学习器称为个体学习器。个体学习器通常由一个现有的学习算法从训练数据产生：

* 只包含同种类型的个体学习器，这样的集成是“同质”的（homogeneous）。同质集成中的个体学习器亦称为”基学习器“（base learner），相应的学习算法称为”基学习算法“（base learning algorithm）。
* 集成也可包含不同类型的个体学习器，这样集成是”异质“的（heterogeneous）。相应的个体学习器，常称为”组件学习器“（component learner）或直接称为个体学习器。<br>

要获得好的集成，个体学习器应“好而不同”。个体学习器要有一定的“准确性”，即学习器不能太坏，而且要有“多样性”（diversity），即学习器之间有差异。事实上，如何产生并结合“好而不同”的个体学习器，恰是集成学习研究的核心。



### 3. boosting  bagging
**Boosting:** 迭代提升，串行，经典例子为AdaBoost。<br>
Boosting是一族可将弱学习器提升为强学习器的算法。这族算法的工作机制类似：先从初始训练集训练出一个基学习器，再根据基学习器的表现对训练样本分布进行调整，使得先前基学习器做错的训练样本在后续收到更多的关注，然后基于调整后的样本分布来训练下一个基学习器；如此重复进行，直到基学习器数目达到事先指定的值T，最终将这T个学习器进行加权结合。<br>
AdaBoost Algorithm ：<br>
1. Input: $N$ samples $(x_1,y_1),...,(x_N,y_N), where\ y_i\in(-1,1)$ and $M$ rounds iterations<br>
2. Initialise weight: $w_{1,i}=1/N$
3. for $m= 1 : M:$
* compute weak classifier $\phi_m(x)$
* compute error $\varepsilon_m$ 
* compute weight of weak classifer $\alpha_m$
* update training set distribution
4. finally the ensemble classifer is: $f(x)=sgn(\sum_{m=1}^M\alpha_m\phi_m(x))$

**Bagging:** 有放回booststrap采样，可并行操作，经典例子为RF。<br>
Bagging可以简单的理解为：有放回抽样，多数表决（分类）或简单平均（回归），同时Bagging的基学习器之间属于并列生成，不存在强依赖关系。<br>
流程：
1. 从原始样本集中抽取训练集。每轮从原始样本集中使用Bootstraping（有放回）的方法抽取n个训练样本（在训练集中，有些样本可能被多次抽取到，而有些样本可能一次都没有被抽中）。共进行k轮抽取，得到k个训练集。（我们这里假设k个训练集之间是相互独立的，事实上不是完全独立）

2. 每次使用一个训练集得到一个模型，k个训练集共得到k个模型，但是是同种模型。（注：k个训练集虽然有重合不完全独立，训练出来的模型因为是同种模型也是不完全独立。这里并没有具体的分类算法或回归方法，我们可以根据具体问题采用不同的分类或回归方法，如决策树、感知器等）

3. 对分类问题：将上步得到的k个模型采用投票的方式得到分类结果；对回归问题，计算上述模型的均值作为最后的结果。（所有模型的重要性相同）

对于Bagging需要注意的是，**每次训练集可以取全部的特征进行训练，也可以随机选取部分特征训练**，例如随机森林就是每次随机选取部分特征。

### 4. 结合策略(平均法，投票法，学习法)
假设集成包含T个基学习器
$\{h_1,h_2,...,h_T\}$，其中$h_i$在样本x上的输出为$h_i(x)$，接下来我们介绍几种对$h_i$进行结合的常见策略。
1. 平均法<br>
对数值型输出$h_i(x)∈R$，最常见的结合策略就是使用平均法(averaging):
* Simple Averaging:
$H(x)=\frac{1}{T}\sum_{i=1}^{T}h_i(x)$
* Weighted Averaging:
$H(x)=\sum_{i=1}^{T}w_ih_i(x)$, where $w_i$ is the weight of base learner $h_i$ ,  
$w_i\geq0$ and $\sum_{i=1}^{T}w_i=1$

简单平均法看起来似乎性能不如加权平均法，但事实上，由于加权平均法的权重一般是直接从训练数据中习得（例如估计出个体学习器的误差，然后令权重大小与误差大小成反比），而现实任务中的训练数据集又往往不够充分或者包含噪音，尤其是对于规模较大的集成来说，要学习的权重也比较多，会比较容易导致过拟合。

因此，实验和应用均表明，加权平均法未必就一定强于简单平均法。一般而言，在个体学习器性能差异较大时宜使用加权平均法，而在个体学习器性能相近时使用简单平均法。
2. 投票法<br>
对分类任务来说，学习器$h_i$将从类别标记集合${c_1,c_2,...,c_N}$中预测出一个label，最常见的集合策略就是使用投票法（voting）。为了方便讨论，我们假设$h_i$在样本x上的预测输出表示为一个N维向量$(h^1_i(x);h^2_i(x);...;h^N_i(x))$，其中
$h^j_i(x)$是$h_i$在类别标记$c_j$上的输出。
* majority voting: 若某类标记票数**超过半数**，则预测为该类别，否则拒绝预测。

* plurality voting: 若某类标记**票数最多**，就预测为该类，若有多个类别一样多，就随机预测一个。
（它与绝对多数投票法不同的有，绝对多数投票法可以做出“不预测”的决定，但是相对多数投票法必然会预测出一个结果）
* weighted voting: 与加权平均法类似。

标准的绝对多数投票法提供了“拒绝预测”的选项，这在对可靠性要求比较高的学习任务中是一个很好的机制。但如果学习任务要求必须提供预测结果，绝对多数投票法就可以退化为相对多数投票法。

在现实任务中，我们还需要给不同类型的个体学习器规定输出格式。因为不同类型的个体学习器可能会产生不同类型的$h^j_i(x)$值，常见的有：
* 类标记：$h^j_i(x)∈{0,1}$，若$h_i$将样本$x$预测为类别$c_j$则取值为1，否则为0。这种使用类标记的投票也称作硬投票(hard voting)。
* 类概率：$h^j_i(x)∈[0,1]$，相当于是对后验概率$P(c_j∣x)$的一个估计。使用类概率的投票也叫软投票(soft voting)。

对一些能在预测出类别标记的同时产生分类置信度的学习器（如SVM），其分类置信度可以转化为类概率使用。但若这类值未进行规范化，像是支持向量机的分类间隔值，就必须使用一些技术如Platt缩放(Platt scailing)、等分回归(isotonic regression)等进行“校准”(calibration)后才能作为类概率使用。

尽管基分类器估计出的类概率值一般都不太准确，但基于类概率进行结合往往要比直接基于类标记进行结合的性能更好。需注意的是，若基学习器的类型不同，其类概率值之间无法进行直接比较；需要将类概率转化为类别标记输出（例如将类概率最大的$h^j_i(x)$设为1，其它设为0）然后再投票。

3. 学习法
当**训练数据很庞大时**，一种更为强大的结合策略就是使用学习法，即通过另一个学习器来进行结合。Stacking就是学习法的典型代表。在这里，我们将个体学习器称为初级学习器，用于结合的学习器称为次级学习器或元学习器(meta-learner)。

Stacking先从初始数据集训练出初级学习器，然后“生成”一个新数据集用于训练次级学习器。在这个新数据集中，初级学习器的输出被当作是样本的输入特征，而初始样本的标记仍然被当作样本标记。我们假定初级学习器使用不同学习算法产生，也就是初级集成是异质的（初级学习器也可以是同质的）。

在训练阶段，次级训练集是利用初级学习器产生的，若是直接用初级学习器的训练集来产生次级训练集，则会有较大的过拟合风险；因此，一般我们会用交叉验证或者留一法这样的方式，用训练初级学习器未使用的样本来产生次级学习器的训练样本。

以k折交叉验证为例，初始训练集D被随机划分为k个大小相似的集合$D_1,...,D_k$，令$D_j$和
$\overline D_j=D-D_j$分别表示第j折的测试集和训练集。假设我们有T个初级学习算法，初级学习器$h^{(j)}_t$通过在$\overline D_j$上使用第t个学习算法得到。对$D_j$中每个样本$x_i$，令$z_{it}=h^{(j)}_t(x_i)，则由$x_i$所产生的次级训练样例的实例部分为$z_i=(z_{i1};z_{i2};...;z_{iT})，标记部分为$y_i$。于是，在整个交叉验证过程结束后，从这T个初级学习器产生的次级训练集是$D′={(z_i,y_i)}^m_{i=1}$，然后$D′$将用于训练次级学习器。

次级学习器的*输入属性表示*和*次级学习算法*对Stacking集成的泛化性能有很大影响。有研究表明，将初级学习器的输出类概率作为次级学习器的输入属性，用**多响应线性回归**(Multi-response Linear Regression,MLR)作为次级学习算法效果较好，若还能在MLR中使用不同的属性集就会更好。MLR是基于线性回归的分类器，它对每个类分别进行线性回归，属于该类的训练样本所对应的输出被置为1，其他类置为0；测试样本将被分给输出值最大的类。

贝叶斯模型平均（Bayes Model Averaging，BMA）基于后验概率来为不同模型赋予权重，可视为加权平均法的一种特殊实现。Clarke对Stacking和BMA进行了比较，理论上说，若数据生成模型恰好在当前考虑的模型中，且数据噪声很少，则BMA不比Stacking差；然而，在现实应用中我们无法保证数据生成模型一定在当前考虑的模型中，甚至可能难以用当前考虑的模型来进行近似，因此，Stacking通常优于BMA，因为其鲁棒性（Robust）比BMA更好，而且BMA对模型近似误差非常敏感。

### 5. 随机森林思想
随机森林（Random Forest ）是Bagging的一个扩展变体。

RF是以决策树为基学习器构建的Bagging集成。但不同的是，RF的基学习器决策树引入了**随机属性选择**：

对于一个传统的决策树，我们对它的节点的每一次划分都会从当前节点的属性集合中（假设有d个属性）选择一个最优属性进行划分；而在RF中，对决策树的每个节点我们会先从节点包含的d个属性中随机的挑选出包含**k个属性的子集**，再**从这个子集中选择一个最优的属性用于划分**，参数k控制了随机性的引入程度：若k=d，则基决策树等同于传统决策树；若k=1，则基决策树的每个节点都是随机选择一个属性用于划分。一般来说，$k=log_2d$是最适合的值。

### 6. 随机森林的推广

由于RF在实际应用中的良好特性，基于RF，有很多变种算法，应用也很广泛，不光可以用于分类回归，还可以用于特征转换，异常点检测等。

**1. extra trees**

extra trees是RF的一个变种, 原理几乎和RF一模一样，仅有区别有：

1） 对于每个决策树的训练集，RF采用的是随机采样bootstrap来选择采样集作为每个决策树的训练集，而extra trees一般不采用随机采样，即每个决策树采用原始训练集。

2） 在选定了划分特征后，RF的决策树会基于基尼系数，均方差之类的原则，选择一个最优的特征值划分点，这和传统的决策树相同。但是extra trees比较的激进，他会随机的选择一个特征值来划分决策树。

从第二点可以看出，由于随机选择了特征值的划分点位，而不是最优点位，这样会导致生成的决策树的规模一般会大于RF所生成的决策树。也就是说，模型的方差相对于RF进一步减少，但是偏倚相对于RF进一步增大。在某些时候，extra trees的泛化能力比RF更好。

**2. Totally Random Trees Embedding**

Totally Random Trees Embedding(以下简称 TRTE)是一种非监督学习的数据转化方法。它将低维的数据集映射到高维，从而让映射到高维的数据更好的运用于分类回归模型。支持向量机中运用了核方法来将低维的数据集映射到高维，此处TRTE提供了另外一种方法。

TRTE在数据转化的过程也使用了类似于RF的方法，建立T个决策树来拟合数据。当决策树建立完毕以后，数据集里的每个数据在T个决策树中叶子节点的位置也定下来了。比如我们有3颗决策树，每个决策树有5个叶子节点，某个数据特征x划分到第一个决策树的第2个叶子节点，第二个决策树的第3个叶子节点，第三个决策树的第5个叶子节点。则x映射后的特征编码为(0,1,0,0,0,  0,0,1,0,0, 0,0,0,0,1), 有15维的高维特征。这里特征维度之间的空格是为了强调三颗决策树各自的子编码。

映射到高维特征后，可以继续使用监督学习的各种分类回归算法了。


**3. Isolation Forest**

Isolation Forest（以下简称IForest）是一种异常点检测的方法。它也使用了类似于RF的方法来检测异常点。

对于在T个决策树的样本集，IForest也会对训练集进行随机采样,但是采样个数不需要和RF一样，对于RF，需要采样到采样集样本个数等于训练集个数。但是IForest不需要采样这么多，一般来说，采样个数要远远小于训练集个数？为什么呢？因为我们的目的是异常点检测，只需要部分的样本我们一般就可以将异常点区别出来了。

对于每一个决策树的建立， IForest采用随机选择一个划分特征，对划分特征随机选择一个划分阈值。这点也和RF不同。

另外，IForest一般会选择一个比较小的最大决策树深度`max_depth`,原因同样本采集，用少量的异常点检测一般不需要这么大规模的决策树。

对于异常点的判断，则是将测试样本点x拟合到T颗决策树。计算在每颗决策树上该样本的叶子节点的深度$h_t(x)$。从而可以计算出平均高度$h(x)$。

### 7. 优缺点
RF的优点：
1. 在数据集上表现良好，相对于其他算法有较大的优势（训练速度、预测准确度）；
2. 能够处理很高维的数据，并且不用特征选择，而且在训练完后，给出特征的重要性；
3. 由于采用了随机采样，训练出的模型的方差小，泛化能力强。
4. 对部分特征缺失不敏感。
5. 可以并行化建树。

RF的缺点：
1. 在噪声较大的分类或者回归问题上回过拟合。 

### 8. sklearn参数


In [0]:
class sklearn.ensemble.RandomForestClassifier(n_estimators=10, criterion=’gini’, 
                                              max_depth=None, min_samples_split=2, 
                                              min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                              max_features='auto', max_leaf_nodes=None,
                                              bootstrap=True, oob_score=False,
                                              n_jobs=1, random_state=None, verbose=0,  
                                              warm_start=False, class_weight=None) 

**Parameters:**

***关于决策树的参数：***

`criterion` : string, optional (default=”gini”)
The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Note: this parameter is tree-specific.

`max_features` : int, float, string or None, optional (default=”auto”)
The number of features to consider when looking for the best split:

If int, then consider max_features features at each split.
If float, then max_features is a fraction and int(max_features * n_features) features are considered at each split.
If “auto”, then max_features=sqrt(n_features).
If “sqrt”, then max_features=sqrt(n_features) (same as “auto”).
If “log2”, then max_features=log2(n_features).
If None, then max_features=n_features.
Note: the search for a split does not stop until ***at least one valid partition of the node samples is found***, even if it requires to effectively inspect more than max_features features.

`max_depth` : integer or None, optional (default=None)
The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

`min_samples_split` : int, float, optional (default=2)
The minimum number of samples required to split an internal node:

If int, then consider min_samples_split as the minimum number.
If float, then min_samples_split is a fraction and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.

`min_samples_leaf` : int, float, optional (default=1)
The minimum number of samples required to be at a leaf node. A split point at any depth will only be considered if it leaves at least min_samples_leaf training samples in each of the left and right branches. This may have the effect of smoothing the model, especially in regression.

If int, then consider min_samples_leaf as the minimum number.
If float, then min_samples_leaf is a fraction and ceil(min_samples_leaf * n_samples) are the minimum number of samples for each node.

`min_weight_fraction_leaf` : float, optional (default=0.)
The minimum weighted fraction of the sum total of weights (of all the input samples) required to be at a leaf node. Samples have equal weight when sample_weight is not provided.

`max_leaf_nodes` : int or None, optional (default=None)
Grow trees with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes.

`min_impurity_decrease` : float, optional (default=0.)

`min_impurity_split` : float, (default=1e-7)
Threshold for early stopping in tree growth. A node will split if its impurity is above the threshold, otherwise it is a leaf.

***随机森林特有的参数：***

`n_estimators` : integer, optional (default=10)
The number of trees in the forest.

bootstrap : boolean, optional (default=True)
Whether bootstrap samples are used when building trees. If False, the whole datset is used to build each tree.

`oob_score` : bool, optional (default=False)
whether to use out-of-bag samples to estimate the R^2 on unseen data. ??

`n_jobs` : int or None, optional (default=None)
The number of jobs to run in parallel for both fit and predict. None means 1 non-parallel. -1 means using all processors.

`random_state` : int, RandomState instance or None, optional (default=None)
If int, random_state is the seed used by the random number generator; If RandomState instance, random_state is the random number generator; If None, the random number generator is the RandomState instance used by np.random.

`verbose` : int, optional (default=0)
Controls the verbosity when fitting and predicting.

`warm_start` : bool, optional (default=False)
When set to True, reuse the solution of the previous call to fit and add more estimators to the ensemble, otherwise, just fit a whole new forest.

***预测形式:***

`predict(X)`
Predict regression target for X.

The predicted regression target of an input sample is computed as the mean predicted regression targets of the trees in the forest.

`predict_proba(X)`：给出带有概率值的结果。每个点在所有label的概率和为1.  

predict(x)：直接给出预测结果。内部还是调用的predict_proba()，根据概率的结果看哪个类型的预测值最高就是哪个类型。  

`predict_log_proba(X)`：和predict_proba基本上一样，只是把结果给做了log()处理

### 9.应用场景

* Sales Forecast- 你能预测公司6周后的Sales吗？

营销人经常会被要求提前预测销量。比如你在一个城市要新开几家新的店铺，每家店铺的sales都受到以下例如因素的影响：地理位置、促销活动、Holidays、seasonality、地域特征等等。你就可以利用随机森林模型来获得每个因素对sales的影响力。

* Email Marketing- 预测用户什么时候Unsubscribe

营销人尤其关心评估过去campaign的表现并以此预测未来的campaign。一个常见的metric就是客户流失率（churn）。可以建立一个有效的分类模型，利用历史数据和机器学习的方法（随机森林）来把客户分成会流失的不会流失的。公司基于模型结果可以用调整他们的email frequency或考虑使用其他传播渠道。

**参考**：

西瓜书

cs229吴恩达机器学习课程

李航统计学习

https://zyzypeter.github.io/2017/08/09/machine-learning-ch13-Bagging-RF/

https://www.cnblogs.com/pinard/p/6156009.html

公式推导参考：http://t.cn/EJ4F9Q0